Temperautre :- fancy term for dividing the logits by a value greater than 0.

And multinomial probability distribution samples next token according to a probability score.

In [2]:
import torch

In [7]:
logits = torch.rand(9)

In [8]:
logits

tensor([0.6658, 0.6068, 0.2864, 0.7993, 0.4788, 0.6706, 0.5592, 0.3304, 0.6525])

In [9]:
next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)

In [10]:
torch.argmax(next_token_logits,dim=-1,keepdim=True)

tensor([3])

In [26]:
temperature1 =0.1
temperature2 =1
temperature3 =3
probas1 = torch.round(torch.softmax(next_token_logits/temperature1,dim=-1)*1000)/1000
probas2 = torch.round(torch.softmax(next_token_logits/temperature2,dim=-1)*1000)/1000
probas3 = torch.round(torch.softmax(next_token_logits/temperature3,dim=-1)*1000)/1000
print(probas1)
print(probas2)
print(probas3)

tensor([0.0000, 0.0000, 0.0000, 0.9910, 0.0000, 0.0000, 0.0000, 0.0090, 0.0000])
tensor([0.0610, 0.0020, 0.0000, 0.5720, 0.0030, 0.0000, 0.0000, 0.3580, 0.0040])
tensor([0.1570, 0.0470, 0.0190, 0.3310, 0.0600, 0.0200, 0.0190, 0.2830, 0.0630])


In [36]:
torch.multinomial(probas1,num_samples=1)

tensor([3])

In [37]:
torch.multinomial(probas2,num_samples=1)

tensor([7])

In [42]:
torch.multinomial(probas3,num_samples=1).item()

8

Top-K Sampling

In [53]:
next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)

In [64]:
topk=3
values,indices = torch.topk(next_token_logits,topk)
print(values,indices)

tensor([6.7500, 6.2800, 4.5100]) tensor([3, 7, 0])


In [62]:
new_logits = torch.where(
    condition=next_token_logits < values[-1],
    input = torch.tensor(float("-inf")),
    other= next_token_logits
)
new_logits

tensor([4.5100,   -inf,   -inf, 6.7500,   -inf,   -inf,   -inf, 6.2800,   -inf])

In [63]:
new_logits = torch.softmax(new_logits,dim=-1)
new_logits

tensor([0.0615, 0.0000, 0.0000, 0.5775, 0.0000, 0.0000, 0.0000, 0.3610, 0.0000])

In [59]:
def generate(idx,context_length,model,max_tokens,temperature=0.0,top_k=None,end_token=None):
  for _ in range(max_tokens):
      new_idx = idx[:,-context_length:]
      with torch.no_grad():
        logits = model(new_idx)
      logits = logits[:,-1,:]

      #temperature scaling + topk sampling
      if top_k is not None:
          new_logits_values,new_logits_idx = torch.topk(logits,top_k)
          logits = torch.where(condition= logits<new_logits_values[:,-1],input = torch.tensor(float("-inf")),other= logits)
      if temperature>0.0:
          logits = logits/temperature
          probas = torch.softmax(logits,dim=-1)
          next_idx = torch.multinomial(probas,num_samples=1)
      else:
          next_idx = torch.argmax(logits,dim=-1,keep_dim=True)
      if next_idx==end_token:
          break
  idx= torch.cat((idx,next_idx),dim=-1)
  return idx


In [ ]:
def generate(idx,context_length,model,max_words,temperature=0.0,top_k=None,end_token=None):
  model.eval()
  for _ in range(max_words):
      new_idx = idx[:,-context_length:]
      with torch.no_grad():
        logits = model(new_idx)
      logits = logits[:,-1,:]

     # temperature scaling + topk sampling
      if top_k is not None:
          top_logits,_ = torch.topk(logits,top_k)
          min_val = top_logits[:,-1].unsqueeze(-1)
          logits = torch.where(logits<min_val,torch.tensor(float("-inf")).to(logits.device),logits)
      if temperature>0.0:
          logits = logits/temperature
          probas = torch.softmax(logits,dim=-1)
          next = torch.multinomial(probas,num_samples=1)
      else:
        logits = torch.softmax(logits,dim=-1)
        next = torch.argmax(logits,dim=-1,keepdim=True)
      if next==end_token:
          break
      idx = torch.cat((idx,next),dim=-1)
  return idx
